In [ ]:
#oversampling, downsampling, Synthetic minority sampling technique (SMOTE)
#구현해보기

#mnist 0 과 1 label만 가져와서 0 label은 1 label 개수의 1/10 만큼만 해준뒤 학습 시켜보도록 하자.

#이후 oversampling과 downsamping, knn을 이용한 SMOTE를 이용해보자.

In [1]:
import pandas as pd
import numpy as np

In [99]:
train = pd.read_csv("../mnist_train.csv")

In [100]:
train_0 = train.loc[train['label']==0]
train_1= train.loc[train['label']==1]

train_0 = train_0.iloc[:500,:]

train = pd.concat([train_0,train_1],axis=0).reset_index().iloc[:,1:]

In [101]:
train_label_1 = train.iloc[:,0]

In [102]:
train_label = train.iloc[:,0]
train = train.iloc[:,1:]

In [104]:
train_1 = train

In [90]:
train = train/255
train = np.array(train).reshape(train.shape[0],28,28,1)
train_label= pd.get_dummies(train_label)

In [60]:
from keras.layers import pooling, Dense, Conv2D, Dropout, Flatten,Activation
from keras.models import Sequential, Model

In [65]:
def model_making():
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation ='relu',input_shape=(28,28,1)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32,(3,3),activation ='relu'))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [94]:
model = model_making()
model.fit(train,train_label)

Epoch 1/1
 928/7242 [==>...........................] - ETA: 28s - loss: 0.0618 - acc: 0.9784

KeyboardInterrupt: 

In [67]:
test = pd.read_csv("../mnist_test.csv")

In [68]:
test_0 = test.loc[test['label']==0]
test_1= test.loc[test['label']==1]

test = pd.concat([test_0,test_1],axis=0).reset_index().iloc[:,1:]

test_label = test.iloc[:,0]
test = test.iloc[:,1:]

test = test/255
test = np.array(test).reshape(test.shape[0],28,28,1)
#test_label= pd.get_dummies(test_label)

In [69]:
res = model.predict(test)
res = np.argmax(res,axis=1)

In [95]:
from sklearn.metrics import accuracy_score

accuracy_score(res,test_label)

0.9985815602836879

In [68]:
#아무 작업도 하지 않았을때 99.85%의 정확도가 나오게 되었다.



ModuleNotFoundError: No module named 'imbalance_learn'

In [96]:
from imblearn.under_sampling import *

In [105]:
X, y = RandomUnderSampler(random_state=0).fit_sample(train_1, train_label_1)
#X= X/255
X = np.array(X).reshape(X.shape[0],28,28,1)
y= pd.get_dummies(y)

In [110]:
model = model_making()
model.fit(X,y)
down_res = model.predict(test)
down_res = np.argmax(down_res,axis=1)

Epoch 1/1
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6740 - acc: 0.9540


In [111]:
accuracy_score(down_res,test_label)

0.9952718676122931

In [ ]:
#약간 정확도가 떨어져, 99.57%이 나왔다. 하지만 학습 데이터 대비 효율성이 올라감.
#일부러 데이터 양을 줄이는 것이므로 약간 비추

In [112]:
from imblearn.over_sampling import *


In [113]:
X, y = RandomOverSampler(random_state=0).fit_sample(train_1, train_label_1)
X= X/255
X = np.array(X).reshape(X.shape[0],28,28,1)
y= pd.get_dummies(y)

In [115]:
model = model_making()
model.fit(X,y)
up_res = model.predict(test)
up_res = np.argmax(down_res,axis=1)

Epoch 1/1
13484/13484 [==============================] - 48s 4ms/step - loss: 0.0101 - acc: 0.9970


In [116]:
accuracy_score(up_res,test_label)

0.9985815602836879

In [ ]:
#같은 정확도가 나오게됨.
#중복인 데이터가 늘어나 비효율적

In [117]:
#복합 샘플링 SMOTEENN = SMOTE +ENN

from imblearn.combine import *

In [118]:
X, y = SMOTEENN(random_state=0).fit_sample(train_1, train_label_1)
X= X/255
X = np.array(X).reshape(X.shape[0],28,28,1)
y= pd.get_dummies(y)

In [119]:
model = model_making()
model.fit(X,y)
combine_res = model.predict(test)
combine_res = np.argmax(combine_res,axis=1)

Epoch 1/1
13483/13483 [==============================] - 49s 4ms/step - loss: 0.0089 - acc: 0.9967


In [120]:
accuracy_score(combine_res,test_label)

0.9985815602836879

In [ ]:
#또 같은결과.
#하지만 둘과 다르게 너무 중복점을만들거나(knn한뒤 점이음느낌) 데이터를 없애지않으므로
#좀더 융통성있는 결과 도출가능